In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

# Read Dataset

In [3]:
train_file_path = '/content/drive/MyDrive/ITU_Competition_Intrusion_and_Vulnerability_Detection_in_Software_Defined_Networks(SDN)/train_data.csv'
test_file_path = '/content/drive/MyDrive/ITU_Competition_Intrusion_and_Vulnerability_Detection_in_Software_Defined_Networks(SDN)/test_data.csv'

train_df = pd.read_csv(train_file_path,skipinitialspace=True)
test_df = pd.read_csv(test_file_path,skipinitialspace=True)

In [4]:
#Split data into X(input features) and Y (labels)
X_train = train_df.drop(train_df.columns[-15:], axis=1)
y_train = train_df[train_df.columns[-15:]]

X_test = test_df.drop(test_df.columns[-15:], axis=1)
y_test = test_df[test_df.columns[-15:]]

In [5]:
print("Records deleted in training dataset:",train_df.isnull().sum().sum())
print("Records deleted in test dataset:", test_df.isnull().sum().sum())

Records deleted in training dataset: 0
Records deleted in test dataset: 0


# Decision Tree

In [8]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)

In [9]:
y_pred = clf.predict(X_test) # change to x_test_0
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9794190092661376


In [10]:
from sklearn.metrics import multilabel_confusion_matrix
m_cm = multilabel_confusion_matrix(y_test, y_pred)
print(m_cm)

In [ ]:
# Plot confusion matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
# Define the class labels
class_labels = y_test.columns

# Create a heatmap using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Random Forest